# 네이버 자주 본 뉴스 크롤링

1) 수집내용

    1) 많이본뉴스–섹션별(정치~IT/과학)Top5기사제목,신문사,뷰 
    2) 해당 기사별 기사 내용, 리액션 (좋아요 ~ 후속기사 원해요)
    
2) 수집방법(택1)

    1) [기본] Requests , BeautifulSoup, Selenium
    2) [심화] Requests, BeautifulSoup (+ 멀티프로세싱)
    
3) 수집범위 및 저장

    1) 2019년7월21일~2020년8월20일(동작가능,실제구동x)
    2) 하나의 파일로 저장 (방식 자유)
    3) Ex)총6섹션*Top5*365일=10950rows

In [411]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from IPython.display import Image
import os
from pathlib import Path
import glob

import datetime
import time

## 날짜 String 생성

In [412]:
start_date = datetime.datetime(2019, 7, 21, 3, 22, 32)
start_date.strftime("%Y%m%d %Y-%m-%d %H:%M:%S") #strftime 활용 예시

'20190721 2019-07-21 03:22:32'

In [413]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [414]:
date_string_list = get_date_string(start_date, 366)

In [415]:
date_string_list[0], date_string_list[-1] # 생성 완료

('20190721', '20200720')

## Part 1. Request를 이용한 크롤링

In [416]:
def get_top_news(date):
    """
    해당 날짜의 자주 본 뉴스 25개에 대한 정보를 반환하는 함수입니다.
    """
    url = f"https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&date={date}" # 이부분을 잘 채워넣어주세요 (네이버 뉴스 자주보는 뉴스 링크에서 date를 제외한 부분)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    result = []
    
    # 신문 제목 & 기사 링크
    s1 = soup.select('dt > a')
    titles = []  # 신문 제목
    links = []  # 기사 링크
    for i in s1:
        titles.append(i.text.strip())
        if 'href' in i.attrs:
            links.append(i.attrs['href'])

    #신문사명 & View
    s2 = soup.select('dl')
    names_views = []
    names = []  # 신문사명
    views = []  # VIew
    for i in s2:
        names_views.append(i.text)
    for i in range(30):
        names.append(names_views[i].split('\n')[-5].strip())
        views.append(int(names_views[i].split('\n')[-3][2:].replace(',', '')))  # 콤마 제거 후 정수형으로 변환

    for i in range(30):
        result.append([titles[i], links[i], names[i], views[i]])
    
    return result

In [417]:
TEST_DATE_RANGE = 20 # 1년치 전체를 하는 것이 아닌 앞 20일치만 테스트 해보기 위함 (개인적으로 바꿔도 무방)

news_data = []
for date in date_string_list[:TEST_DATE_RANGE]:
     news_data.extend(get_top_news(date)) # 결과로 나온 30개를 이어 붙임

In [418]:
len(news_data)

600

In [419]:
news_data[:5] # 결과 예시

[['[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=032&aid=0002952683&date=20190721&type=1&rankingSectionId=100&rankingSeq=1',
  '경향신문',
  201039],
 ['조국 폭풍페북, 日주장 정면반박…"친일파" 표현은 野 반발',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=008&aid=0004251344&date=20190721&type=1&rankingSectionId=100&rankingSeq=2',
  '머니투데이',
  185396],
 ['조국, 연일 對日 \'항전\' 주문…"겁먹고 쫄지말자…싸워 이겨…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0010969325&date=20190721&type=1&rankingSectionId=100&rankingSeq=3',
  '연합뉴스',
  130198],
 ['[김순덕의 도발]복수를 하려면 아일랜드처럼!',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=020&aid=0003230442&date=20190721&type=1&rankingSectionId=100&rankingSeq=4',
  '동아일보',
  120897],
 ['조국, 또 페북에 反日 선전전..."文정부, 서희·이순신 역할…',
  '/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003462170&date=20190721&type=1&rankingSectionId=100&rank

In [420]:
# 결과물을 데이터 프레임으로 변환 및 Column Name 부여
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])

In [421]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,경향신문,201039
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,머니투데이,185396
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,130198
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,동아일보,120897
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,조선일보,119463
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,서울경제,173703
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,134014
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,연합뉴스,111786
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,뉴스1,77278


In [422]:
# URL 앞에  "https://news.naver.com" 붙이기
df_top_news.url = "https://news.naver.com"+df_top_news.url

In [423]:
df_top_news

,title,url,press,views
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,201039
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,185396
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,130198
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,120897
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,119463
...,...,...,...,...
595,"갤노트10 만져본 미국인들 ""실물 보고싶어 일부러 찾아와""",https://news.naver.com/main/ranking/read.nhn?m...,서울경제,173703
596,"""유튜브 뛰어드니""…월급 295만→536만원 '껑충' vs 소득 '…",https://news.naver.com/main/ranking/read.nhn?m...,뉴스1,134014
597,"[SNS 세상] ""공감이 최고의 위로"" 암 투병기 연재하는 뷰티…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,111786
598,네이버·카카오 2분기 성적표 '희비'…하반기 '페이전쟁' 격…,https://news.naver.com/main/ranking/read.nhn?m...,뉴스1,77278


## Part 2. Selenium을 이용한 크롤링

In [424]:
driver_path = r'C:\Users\tpdud\Desktop\Tobigs\week5_Crawling\chromedriver.exe'

In [425]:
driver = webdriver.Chrome(driver_path) # Chrome driver 실행

In [426]:
df_top_news.url[:10]

0    https://news.naver.com/main/ranking/read.nhn?m...
1    https://news.naver.com/main/ranking/read.nhn?m...
2    https://news.naver.com/main/ranking/read.nhn?m...
3    https://news.naver.com/main/ranking/read.nhn?m...
4    https://news.naver.com/main/ranking/read.nhn?m...
5    https://news.naver.com/main/ranking/read.nhn?m...
6    https://news.naver.com/main/ranking/read.nhn?m...
7    https://news.naver.com/main/ranking/read.nhn?m...
8    https://news.naver.com/main/ranking/read.nhn?m...
9    https://news.naver.com/main/ranking/read.nhn?m...
Name: url, dtype: object

In [427]:
NEWS_TEST_RANGE = 10
for idx, news_url in enumerate(df_top_news.url[:NEWS_TEST_RANGE]):
    # 드라이버 내에서 해당 URL로 이동
    res = driver.get(news_url)

    # 기사 내용
    s1 = driver.find_elements_by_css_selector('#articleBodyContents')
    content = []  # 기사 내용
    for i in s1:content.append(i.text)

    # 리액션
    s2 = driver.find_elements_by_css_selector('#spiLayer > div._reactionModule.u_likeit')
    like = ''  
    for i in s2:like += i.text.replace(',','')
    
    # 예시로 content라는 변수에 기사 내용을 담고 Column "content"에 해당 내용 저장
    df_top_news.loc[idx,"content"] = content
    df_top_news.loc[idx,"good"] = int(like.split('\n')[1])
    df_top_news.loc[idx,"warm"] = int(like.split('\n')[3])
    df_top_news.loc[idx,"sad"] = int(like.split('\n')[5])
    df_top_news.loc[idx,"angry"] = int(like.split('\n')[7])
    df_top_news.loc[idx,"followup"] = int(like.split('\n')[9])

In [428]:
df_top_news.head(20)

,title,url,press,views,content,good,warm,sad,angry,followup
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,201039,중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대회 금상\n\n...,0.0,0.0,0.0,0.0,0.0
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,185396,"[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 해설 글 ""대통령 법률...",1710.0,32.0,13.0,6058.0,48.0
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,130198,"""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""\n\n""文정부 매도 ...",1799.0,19.0,13.0,9120.0,30.0
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,120897,"친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정부...",3017.0,19.0,16.0,623.0,37.0
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,119463,"""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""\n\n""법적·...",374.0,13.0,14.0,11468.0,38.0
5,[취재파일] 日 경제 보복…이재용의 침묵과 최태원의 당당함,https://news.naver.com/main/ranking/read.nhn?m...,SBS,157764,경제보복 노출 기업들엔 경영계 리더 '한 마디' 절실\n\n삼성전자 이건희 회장은 ...,154.0,9.0,63.0,1171.0,49.0
6,"불매운동에 놀란 유니클로 대표 ""추가 사과할 것""",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,136808,"[머니투데이 조성훈 기자] [20일 롯데그룹 사장단 회의뒤 기자들과 만나 ""오해있고...",45.0,27.0,20.0,2921.0,49.0
7,"일본 여행 갔다고 '공개 망신'…""성숙한 불매운동 필요""",https://news.naver.com/main/ranking/read.nhn?m...,SBS,126836,동영상 뉴스\n<앵커>\n\n불매운동이 크게 번지고 있지요. 누가 하자고 해서가 아...,371.0,41.0,29.0,3368.0,61.0
8,"""예·적금 가입 서두르세요""···2%대 이자 사라진다",https://news.naver.com/main/ranking/read.nhn?m...,서울경제,125516,이르면 다음주 중 예·적금 금리 0.1~0.3%p 인하 가능성\n이주열 한국은행 총...,16.0,6.0,5.0,545.0,10.0
9,'주식투자로 50평 아파트' 70대 노부인의 행동재무학 재테크,https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,122556,[머니투데이 강상규 소장] [편집자주] 투자자들의 비이성적 행태를 알면 초과수익을 ...,174.0,10.0,5.0,202.0,9.0


> 본 데이터 프레임은 예시로, 리액션 데이터를 담지 않았습니다. **과제에는 리액션 데이터가 반드시 담겨져있어야합니다.**

1년치 전체를 크롤링하려면 위 코드만으로는 돌아가지 않는다. 연예나 스포츠 기사의 경우 태그가 다르고, 존재하지 않는 페이지도 있다. 위 코드들을 기반으로 하되 예외 사항들을 고려하여 1년치 전체를 가져오는 코드를 아래에 작성하였다.

# 데이터 저장

> 파일 형태로 크롤링한 데이터를 저장

In [446]:
news_data = []
for date in date_string_list:  # 1년치 전체 날짜를 돌아가며 크롤링
     news_data.extend(get_top_news(date))
df_top_news = pd.DataFrame(news_data, columns=["title", "url", "press", "views"])
df_top_news.url = "https://news.naver.com"+df_top_news.url

# 대부분 좋아요~후속기사 원해요만 존재하나, 몇몇 기사의 경우 리액션 카테고리가 다르므로 모두 칼럼으로 만들어 0으로 초기화
df_top_news.loc[:,'좋아요'] = [0] * len(df_top_news)
df_top_news.loc[:,'훈훈해요'] = [0] * len(df_top_news)
df_top_news.loc[:,'슬퍼요'] = [0] * len(df_top_news)
df_top_news.loc[:,'화나요'] = [0] * len(df_top_news)
df_top_news.loc[:,'후속기사 원해요'] = [0] * len(df_top_news)
df_top_news.loc[:,'응원해요'] = [0] * len(df_top_news)
df_top_news.loc[:,'축하해요'] = [0] * len(df_top_news)
df_top_news.loc[:,'기대해요'] = [0] * len(df_top_news)
df_top_news.loc[:,'놀랐어요'] = [0] * len(df_top_news)


for idx, news_url in enumerate(df_top_news.url):
    
    res = driver.get(news_url)
    
    # 기사 내용
    s1 = driver.find_elements_by_css_selector('#articleBodyContents')
    content = []
    for i in s1:content.append(i.text)
    
    if content==[]:
        s1 = driver.find_elements_by_css_selector('#articeBody')  # 위 태그가 아닌 경우 (대부분 이 태그)
        for i in s1:content.append(i.text)
    
    
    # 리액션
    s2 = driver.find_elements_by_css_selector('div._reactionModule.u_likeit > ul > li')
    like = [i.text.split('\n') for i in s2[-6:]]  
    
        
    if content==[]:continue  # 위 태그 이외에 기타 태그이거나 존재하지 않는 페이지일 경우 continue
        
    
    df_top_news.loc[idx,"content"] = content
    
    # 응원해요~놀랐어요가 하나도 없는 거 알았으면 반복문 안하고 그냥 할 수 있었는데... 매우 비효율적이었으나 없을 줄 몰랐기 때문에..
    for i in range(len(like)):
        if like[i][0]=='':continue
        elif like[i][0]=='좋아요':df_top_news.loc[idx,'좋아요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='훈훈해요':df_top_news.loc[idx,'훈훈해요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='슬퍼요':df_top_news.loc[idx,'슬퍼요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='화나요':df_top_news.loc[idx,'화나요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='후속기사 원해요':df_top_news.loc[idx,'후속기사 원해요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='응원해요':df_top_news.loc[idx,'응원해요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='축하해요':df_top_news.loc[idx,'축하해요'] = int(like[i][1].replace(',', ''))
        elif like[i][0]=='기대해요':df_top_news.loc[idx,'기대해요'] = int(like[i][1].replace(',', ''))
        else:df_top_news.loc[idx,'놀랐어요'] = int(like[i][1].replace(',', ''))
    
    
df_top_news

,title,url,press,views,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,응원해요,축하해요,기대해요,놀랐어요,content
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,201039,131,16,17,2605,121,0,0,0,0,중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대회 금상\n\n...
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,185396,1710,32,13,6058,48,0,0,0,0,"[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 해설 글 ""대통령 법률..."
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,130198,1799,19,13,9120,30,0,0,0,0,"""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""\n\n""文정부 매도 ..."
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,120897,3017,19,16,623,37,0,0,0,0,"친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정부..."
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,119463,374,13,14,11468,38,0,0,0,0,"""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""\n\n""법적·..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,'문재인 내려와' '문재인 힘내세요' 전쟁의 승자는?,https://news.naver.com/main/ranking/read.nhn?m...,조선일보,476075,25202,138,65,1227,165,0,0,0,0,부동산에 화난 민심 '실검 챌린지'\n20일 오후 6시 1위 오르자\n문대통령 지지...
10976,"""허가 안났지만 9월부터 본격 생산""…코로나 치료제 승부수 …",https://news.naver.com/main/ranking/read.nhn?m...,한국경제,184355,1258,15,9,579,48,0,0,0,0,서정진 회장 온라인 간담회\n\n연말께 임상 2상 시험 끝내고\n긴급 사용승인 나면...
10977,성공 모델서 실패 사례로…반 토막 난 ‘배달의명수’ [IT선…,https://news.naver.com/main/ranking/read.nhn?m...,헤럴드경제,157583,118,15,23,573,22,0,0,0,0,2달간 MAU 절반 아래로 감소\n관리 소홀에 잦은 시스템 오류\n업계 “대표적 공...
10978,보너스 5000만원·한달 휴가…요즘 기업 '그들' 모시기 경쟁,https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,154136,125,7,6,71,16,0,0,0,0,"[머니투데이 이진욱 기자] [""비대면 시대, 5년간 3.2만명 인력부족""…기업들 A..."


In [455]:
sum(df_top_news.iloc[:,4:12].sum(axis=1)==0)  # 리액션이 제대로 크롤링되지 않은 행의 수

52

In [458]:
df_top_news.content.isnull().sum()  # content가 제대로 크롤링되지 않은 행의 수

3

In [467]:
# 응원해요~놀랐어요는 데이터가 존재하지 않으므로(모두 0, 카테고리에는 있었지만 그 리액션을 누른 사람은 없었다는 것을 의미)
# 현재로서는 무의미한 칼럼이므로 제거
naver_top_news = df_top_news.drop(['응원해요', '축하해요', '기대해요', '놀랐어요'], axis=1)
naver_top_news

,title,url,press,views,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,content
0,"[단독] 황교안 딸 운영 사이트, 대학 진학 후 왜 문 닫았나",https://news.naver.com/main/ranking/read.nhn?m...,경향신문,201039,131,16,17,2605,121,중 3 때 오빠와 장관상이어 고3 때도 ‘장함모’ 활동으로 자원봉사대회 금상\n\n...
1,"조국 폭풍페북, 日주장 정면반박…""친일파"" 표현은 野 반발",https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,185396,1710,32,13,6058,48,"[머니투데이 김성휘 ,백지수 기자] [[the300]징용판결 해설 글 ""대통령 법률..."
2,"조국, 연일 對日 '항전' 주문…""겁먹고 쫄지말자…싸워 이겨…",https://news.naver.com/main/ranking/read.nhn?m...,연합뉴스,130198,1799,19,13,9120,30,"""文정부, 서희와 이순신 역할 함께 수행…지레 겁먹지 말아야""\n\n""文정부 매도 ..."
3,[김순덕의 도발]복수를 하려면 아일랜드처럼!,https://news.naver.com/main/ranking/read.nhn?m...,동아일보,120897,3017,19,16,623,37,"친일잔재를 청산하고 한번도 경험하지 못한 나라로 가는 것이 목적이라면, 문재인 정부..."
4,"조국, 또 페북에 反日 선전전...""文정부, 서희·이순신 역할…",https://news.naver.com/main/ranking/read.nhn?m...,조선일보,119463,374,13,14,11468,38,"""문재인 정부, 국익 수호 위해 '서희' '이순신' 역할 함께 수행""\n\n""법적·..."
...,...,...,...,...,...,...,...,...,...,...
10975,'문재인 내려와' '문재인 힘내세요' 전쟁의 승자는?,https://news.naver.com/main/ranking/read.nhn?m...,조선일보,476075,25202,138,65,1227,165,부동산에 화난 민심 '실검 챌린지'\n20일 오후 6시 1위 오르자\n문대통령 지지...
10976,"""허가 안났지만 9월부터 본격 생산""…코로나 치료제 승부수 …",https://news.naver.com/main/ranking/read.nhn?m...,한국경제,184355,1258,15,9,579,48,서정진 회장 온라인 간담회\n\n연말께 임상 2상 시험 끝내고\n긴급 사용승인 나면...
10977,성공 모델서 실패 사례로…반 토막 난 ‘배달의명수’ [IT선…,https://news.naver.com/main/ranking/read.nhn?m...,헤럴드경제,157583,118,15,23,573,22,2달간 MAU 절반 아래로 감소\n관리 소홀에 잦은 시스템 오류\n업계 “대표적 공...
10978,보너스 5000만원·한달 휴가…요즘 기업 '그들' 모시기 경쟁,https://news.naver.com/main/ranking/read.nhn?m...,머니투데이,154136,125,7,6,71,16,"[머니투데이 이진욱 기자] [""비대면 시대, 5년간 3.2만명 인력부족""…기업들 A..."


리액션이나 content가 제대로 크롤링되지 않은 경우는 약 55케이스 정도라고 파악할 수 있다. 결측치를 최대한 배제하고 어느 정도 크롤링이 잘 되었다고 판단한다.

In [ ]:
# naver_top_news.to_csv()